In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [6]:
# Visit the URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [7]:
# Parse Results HTML with BeautifulSoup
# Find Everything Inside:
#   <ul class="item_list">
#     <li class="slide">

html = browser.html
soup_nasa = BeautifulSoup(html, "html.parser")
required_list = soup_nasa.select_one("ul.item_list li.slide")


In [10]:
print(required_list.prettify())

<li class="slide">
 <div class="image_and_description_container">
  <a href="/news/8830/celebrate-the-perseverance-rover-landing-with-nasas-student-challenge/" target="_self">
   <div class="rollover_description">
    <div class="rollover_description_inner">
     The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.
    </div>
    <div class="overlay_arrow">
     <img alt="More" src="/assets/overlay-arrow.png"/>
    </div>
   </div>
   <div class="list_image">
    <img alt="Collage for mission to Mars" src="/system/news_items/list_view_images/8830_C1-marsworkshop_web-320x240.jpg"/>
   </div>
   <div class="bottom_gradient">
    <div>
     <h3>
      Celebrate the Perseverance Rover Landing With NASA's Student Challenge
     </h3>
    </div>
   </div>
  </a>
  <div class="list_text">
   <div class="list_date">
    January  8, 2021
   </div>
   <div class="content_tit

In [8]:
required_list.find("div", class_="content_title")

<div class="content_title"><a href="/news/8830/celebrate-the-perseverance-rover-landing-with-nasas-student-challenge/" target="_self">Celebrate the Perseverance Rover Landing With NASA's Student Challenge</a></div>

In [11]:
# Scrape the Latest News Title
title = required_list.find("div", class_="content_title").get_text()
print(title)

Celebrate the Perseverance Rover Landing With NASA's Student Challenge


In [12]:
# Scrape the Latest Paragraph Text
title_paragraph = required_list.find("div", class_="article_teaser_body").get_text()
print(title_paragraph)

The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.


In [13]:
# Visit the URL for JPL Featured Space Image 
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [16]:
# Parse Results HTML with BeautifulSoup
jpl_html = browser.html
soup_image = BeautifulSoup(jpl_html, "html.parser")

In [19]:
img_url = soup_image.select_one("background-image").get("src")
img_url

AttributeError: 'NoneType' object has no attribute 'get'